# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL, 
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'home page', 'url': 'https://huggingface.co'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'company page', 'url': 'https://github.com/huggingface'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [12]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
6 days ago
•
278k
•
2.68k
microsoft/VibeVoice-Realtime-0.5B
Updated
2 days ago
•
131k
•
813
zai-org/GLM-4.6V-Flash
Updated
5 days ago
•
67.7k
•
404
mistralai/Devstral-Small-2-24B-Instruct-2512
Updated
2 days ago
•
15.8k
•
323
zai-org/GLM-4.6V
Updated
5 days ago
•
3.37k
•
296
Browse 1M+ models
Spaces
Running
on
Zero
546
Z Image Turbo
🖼
546
Generate stunning images from text prompts
Running
on
Zero
MCP
Featured
1.36k
Z Image Turbo
🏃
1.36k
Generate images from text prompts
Running
on
Zero
MCP
119
Qwen Im

In [20]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n6 days ago\n•\n278k\n•\n2.68k\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\n2 days ago\n•\n131k\n•\n813\nzai-org/GLM-4.6V-Flash\nUpdated\n5 days ago\n•\n67.7k\n•\n404\nmistralai/Devstral-Small-2-24B-Instruct-2512\nUpdated\n2 days ago\n•\n15.8k\n•\n323\nzai-org/GLM-4.6V\nUpdated\n5 days ago\n•\n3.37k\n•\n296\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n546

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the leading AI community platform building the future of machine learning. It serves as a vibrant collaboration hub where researchers, developers, and organizations worldwide come together to create, share, and improve machine learning models, datasets, and applications.

Hugging Face empowers the next generation of machine learning engineers, scientists, and users by providing an open, ethical, and accessible platform to explore AI technologies across various modalities including text, images, video, audio, and even 3D data.

---

## What We Offer

- **Model Hub:** Access and contribute to over **1 million publicly available machine learning models**, keeping pace with the latest advancements in AI.
- **Datasets:** Discover and share **250,000+ curated datasets** fueling groundbreaking ML research and applications.
- **Spaces:** Build and showcase your ML applications on hosted environments for instant use and community feedback.
- **Community:** Join an ever-growing, inclusive community that actively collaborates, shares knowledge, and fosters innovation.
- **Enterprise Solutions:** Robust paid compute and enterprise-grade solutions designed for teams looking to accelerate AI development responsibly.

---

## Our Culture and Community

Hugging Face is more than just a platform — it's a global community emphasizing openness, collaboration, and ethical AI. We strive to make machine learning accessible and fast for everyone by:

- Encouraging sharing and open source contributions.
- Supporting the ethical development and deployment of AI.
- Building tools that cater to all skill levels, from beginners to experts.
- Valuing transparency, inclusivity, and mutual support within our diverse user base.

---

## Our Users and Customers

Our platform serves a wide spectrum of AI enthusiasts including:

- Independent machine learning engineers and researchers building portfolios and showcasing their work.
- Academic institutions and students leveraging the platform for education and experimentation.
- Startups and large enterprises building AI products using our open source models and enterprise compute services.
- Developers creating AI-powered applications across industries such as healthcare, finance, entertainment, and more.

---

## Careers at Hugging Face

Join a passionate team at the forefront of AI innovation. Hugging Face offers opportunities to work in a dynamic, cutting-edge environment focused on:

- Open source collaboration.
- Machine learning research and engineering.
- Building scalable platforms and infrastructure.
- Ethical AI development and responsible technology.

If you are driven by creating the future of AI and want to make a meaningful impact in a community-oriented company, Hugging Face welcomes your talent.

---

## Connect & Explore

- Visit [huggingface.co](https://huggingface.co) to explore models, datasets, and AI applications.
- Sign up to build your ML portfolio and collaborate with thousands globally.
- Learn more about enterprise solutions designed to empower your team’s AI projects.

---

**Hugging Face — The AI community building the future.**  
Open. Collaborative. Ethical. Fast.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is an innovative AI community and platform dedicated to building the future of machine learning (ML). It is the go-to hub where the global ML community collaborates on models, datasets, and applications, enhancing the way AI solutions are developed and deployed.

The platform hosts over **1 million models**, **250,000 datasets**, and **400,000+ AI applications**, empowering users to explore AI across all modalities — including text, image, video, audio, and even 3D content.

---

## Platform Highlights

- **Models:** Access and contribute to a vast repository of pretrained models for diverse use cases, from text generation and speech synthesis to image creation and video processing.
- **Datasets:** Utilize and share extensive datasets curated by the community and industry leaders to train, validate, and benchmark AI models.
- **Spaces:** Launch and showcase AI applications easily with Spaces, where developers run interactive ML demos and apps.
- **Community:** Join a vibrant and collaborative AI ecosystem, from hobbyists to industry experts, sharing knowledge, tools, and inspiration.
- **Open Source Stack:** Build faster and better using Hugging Face’s open-source tools, enabling smooth integration and customization.
- **Enterprise Solutions:** Benefit from scalable team and enterprise plans with advanced security features such as Single Sign-On (SSO), audit logs, controlled data regions, and dedicated support.

---

## Company Culture

Hugging Face thrives on community collaboration, openness, and innovation. It fosters a culture where contributors—from researchers and engineers to creatives—can build, share, and advance ML technologies together. The company promotes transparency through its open-source projects and encourages building a personal ML portfolio, fueling continuous learning and growth.

---

## Customers

Hugging Face serves a broad range of customers, including:

- AI researchers and developers seeking powerful models and datasets
- Enterprises requiring secure, scalable AI platforms with team collaboration tools
- Startups and individual creators building innovative AI applications
- Educational institutions and data scientists exploring cutting-edge machine learning capabilities

---

## Careers & Opportunities

Join Hugging Face to be part of a future-facing AI community that values collaboration, creativity, and impact. The company offers opportunities to work with:

- Machine learning engineers and researchers developing state-of-the-art models
- Software developers contributing to scalable, open-source infrastructure
- Product experts shaping user-friendly ML solutions and enterprise offerings
- Community managers fostering an engaged, global AI network

Check their website for current job openings and become part of a team building the future of AI.

---

## Why Choose Hugging Face?

- **Unmatched Collaboration:** Work alongside the largest AI community to accelerate learning and innovation.
- **Extensive Resources:** Leverage an unparalleled collection of models, datasets, and AI applications.
- **Flexible Solutions:** From free open-source tools to enterprise-grade services with enhanced security.
- **Future-Driven:** Focused on shaping the future of machine learning with a community-centric approach.

---

## Get Started

Explore AI applications or browse over a million models on the Hugging Face platform today.

**Website:** [huggingface.co](https://huggingface.co)  
**Sign Up:** Create your profile and start collaborating!  
**Enterprise:** Contact their team to scale AI initiatives securely across your organization.

---

Build, share, and innovate with the home of machine learning — **Hugging Face**.  
*The AI community building the future.*

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Welcome to Hugging Face  
*Where AI isn’t just artificial, it’s community-powered magic!*

---

## Who We Are  
Hugging Face is the buzzing hive of machine learning where thousands of AI enthusiasts, scientists, and engineers build the future — one model, one dataset, and one witty commit at a time. We’re not just a platform; we’re a vibrant **AI community** dedicated to making machine learning open, ethical, and downright accessible. Think of us as your friendly neighborhood AI collaborators, minus the spandex and capes!

---

## What We Offer  
- **1,000,000+ Models:** Whether you want to generate images from text prompts, get real-time speech synthesis, or create videos from pictures, our vast model repository has your back. Trending models like *Tongyi-MAI/Z-Image-Turbo* and *microsoft/VibeVoice-Realtime-0.5B* are just the tip of the AI iceberg.  
- **250,000+ Datasets:** Feed your algorithms with everything from open medical reasoning data to complex NLP datasets—freshly updated and community-curated.  
- **Spaces:** Deploy your apps and demos directly on the platform, and watch as thousands discover your machine learning wizardry live—because nothing says “look Ma, I made AI!” like sharing your work with the world.  
- **Enterprise & Compute:** Power up your team with our paid solutions that scale like your morning coffee consumption—relentlessly!  

---

## Culture: Hugging is Our Middle Name  
- **Open:** We champion open-source, open minds, and open collaboration.  
- **Inclusive:** Everyone is welcome to join the party—from seasoned AI wizards to curious newcomers.  
- **Innovative:** Our science team is not just on the cutting-edge; they’re sharpening the knife every day.  
- **Fun:** We foster a vibrant community where sharing cool projects, ideas, and even the occasional AI pun happens all the time. (Try saying “machine learning” three times fast!)  

---

## Our Community  
With over **73,000 active members** and counting, Hugging Face is the playground for the AI curious and the masters of machine learning alike. People publish new spaces daily, fork models, and push the boundaries of what open-source AI can do—sometimes even before their morning espresso kicks in.  

---

## Careers: Do You *Hug* AI?  
We’re on a mission to democratize good ML “one commit at a time.” If you want to build an open and ethical AI future, grow with a world-class team of 191+ brilliant minds, and feel the thrill of tech that *actually* changes the world, come join us!  

- **Roles:** Engineers, researchers, community managers, product wizards, and yes—even AI pun masters.  
- **Benefits:** A culture where curiosity is king and collaboration is queen.  

Curious? We thought so. Check out our [Careers page](https://huggingface.co/careers) and find your place among the AI revolutionaries.  

---

## Why Hugging Face?  
Because the future of AI isn’t a solo gig—it’s a collective adventure. We unite the best open-source tools and the sharpest minds under one roof (or cloud), empowering you to explore every modality: text, image, video, audio, even 3D!

Ready to accelerate your machine learning journey and maybe even teach your robot to tell jokes?  

**Start exploring Hugging Face today!**

[Sign up now and join the joyful sharing!](https://huggingface.co/join)  

---

### Hugging Face Colors  
Bright, bold, and a little bit happy:  
- Yellow #FFD21E  
- Orange #FF9D00  
- Gray #6B7280  

Because AI should be sunny, spicy, and smart!  

---

*Hugging Face* – Building the future with a smile, one model at a time.  
Follow us: [GitHub](https://github.com/huggingface) | [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://www.linkedin.com/company/huggingface) | [Discord](https://discord.gg/huggingface)  

---

*Disclaimer: Hugging Face bears no responsibility for side effects like AI addiction, improved productivity, or uncontrollable excitement over ML breakthroughs.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>